In [1]:
#Classification model
#Import dependencies
import numpy as np
import pandas as pd
from skimage.data import imread
import matplotlib.pyplot as plt

In [2]:
import os
print(os.listdir("./"))

['train', 'classifcation-Copy1.ipynb', 'TF KERAS.ipynb', 'sample_submission_v2.csv', 'Segmentation-with-classificatin-code-version-2.ipynb', 'f.txt', 'test', 'linear -> 5 layer network Pandas processing.ipynb', 'segmentation.ipynb', 'submissionv2.csv', 'submissionv3_1.csv', 'segmentation-simple.ipynb', '.ipynb_checkpoints', 'classifcation.ipynb', 'Decode RLE.ipynb', 'CNN_CLASSIFIER', 'train_v2.zip', 'classifcation-Copy2.ipynb', 'linear -> 5 layer network example-Copy1.ipynb', 'classifcation-modify-logits.ipynb', 'submissionv4_1.csv', 'Segmentation-with-classificatin-code-version.ipynb', 'train_ship_segmentations_v2.csv', 'explore.ipynb', 'test_v2.zip', 'TF KERAS-Copy1.ipynb', 'misc']


In [3]:
train_v2 = os.listdir('./train')
print(len(train_v2))

test_v2 = os.listdir('./test')
print(len(test_v2))

192556
15607


In [4]:
#Define Run length encoding and decoding functions
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# ref: https://www.kaggle.com/paulorzp/run-length-encode-and-decode
def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.float32)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction

In [5]:
#Import mask labels
masks = pd.read_csv('train_ship_segmentations_v2.csv')
masks.head()

,ImageId,EncodedPixels
0,00003e153.jpg,NaN
1,0001124c7.jpg,NaN
2,000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...
3,000194a2d.jpg,360486 1 361252 4 362019 5 362785 8 363552 10 ...
4,000194a2d.jpg,51834 9 52602 9 53370 9 54138 9 54906 9 55674 ...


In [6]:
type(masks['EncodedPixels'][3])

str

In [7]:
train = masks
train.head()

,ImageId,EncodedPixels
0,00003e153.jpg,NaN
1,0001124c7.jpg,NaN
2,000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...
3,000194a2d.jpg,360486 1 361252 4 362019 5 362785 8 363552 10 ...
4,000194a2d.jpg,51834 9 52602 9 53370 9 54138 9 54906 9 55674 ...


In [8]:
train['exist_ship'] = train['EncodedPixels'].fillna(0)
train.loc[train['exist_ship']!=0, 'exist_ship']=1
del train['EncodedPixels']

In [9]:
train.head()

,ImageId,exist_ship
0,00003e153.jpg,0
1,0001124c7.jpg,0
2,000155de5.jpg,1
3,000194a2d.jpg,1
4,000194a2d.jpg,1


In [10]:
print(len(train['ImageId']))
print(train['ImageId'].value_counts().shape[0])
train_gp = train.groupby('ImageId').sum().reset_index()
train_gp.loc[train_gp['exist_ship']>0,'exist_ship']=1

231723
192556


In [11]:
len(train_gp)

192556

In [12]:
train_gp.head()

,ImageId,exist_ship
0,00003e153.jpg,0
1,0001124c7.jpg,0
2,000155de5.jpg,1
3,000194a2d.jpg,1
4,0001b1832.jpg,0


In [13]:
print(train_gp['exist_ship'].value_counts())
train_gp= train_gp.sort_values(by='exist_ship')
train_gp = train_gp.drop(train_gp.index[0:100000])

0    150000
1     42556
Name: exist_ship, dtype: int64


In [14]:
print(train_gp['exist_ship'].value_counts())

0    50000
1    42556
Name: exist_ship, dtype: int64


In [15]:
#Use a sample of 1000 (Takes too long to run more than 1000)
train_sample = train_gp.sample(1000)
print(train_sample['exist_ship'].value_counts())
print (train_sample.shape)

0    523
1    477
Name: exist_ship, dtype: int64
(1000, 2)


In [16]:
train_sample.head()

,ImageId,exist_ship
42754,38fa58df2.jpg,0
124027,a4f5599c6.jpg,1
124941,a62e5a47b.jpg,1
14796,139bb7c22.jpg,0
72171,5ff81ec6f.jpg,0


In [32]:
train_sample['ImageId']

42754     38fa58df2.jpg
124027    a4f5599c6.jpg
124941    a62e5a47b.jpg
14796     139bb7c22.jpg
72171     5ff81ec6f.jpg
8914      0bd17ce85.jpg
138978    b8a3fad52.jpg
185591    f6c80e299.jpg
107740    8f4603598.jpg
180171    efa1ebc00.jpg
170961    e362586e4.jpg
124985    a63abb572.jpg
155199    ce5b28150.jpg
19062     194f14e97.jpg
77922     678f81f5c.jpg
93242     7beea573c.jpg
25751     2251c909f.jpg
69349     5c3948f80.jpg
172234    e5151ba14.jpg
52356     45b8927c0.jpg
19123     1964255d5.jpg
57649     4cdb1f108.jpg
54519     4897770dd.jpg
58925     4e8dfdf3b.jpg
83194     6e7ee80b0.jpg
30885     28ff59eb0.jpg
191941    ff2a99919.jpg
148397    c55832cba.jpg
81503     6c403aeb4.jpg
48838     4115fb077.jpg
              ...      
41137     36e449ebb.jpg
16946     168708696.jpg
63997     5537f2728.jpg
66547     589f736d0.jpg
39682     34e777259.jpg
70750     5e1dcd17f.jpg
9052      0bfbdb9bb.jpg
4245      05ab4c9c1.jpg
70218     5d643ffcc.jpg
43755     3a59ce441.jpg
64213     5580d5

# Resize Image

In [20]:
#Import Images
from PIL import Image
data = np.empty((len(train_sample['ImageId']),256,256,3), dtype=np.float32)
data_target = np.empty((len(train_sample['ImageId'])), dtype=np.int32)
image_name_list = train_v2
index = 0
for image_name in image_name_list:
    if image_name in list(train_sample['ImageId']):
        imageA = Image.open('./train/' + image_name).resize((256,256)).convert('RGB')
        data[index]=imageA
        data_target[index]=train_sample[train_gp['ImageId'].str.contains(image_name)]['exist_ship'].iloc[0]
        index+=1

/home/jun_inthebay/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


KeyboardInterrupt: 

In [30]:
type(data)

numpy.ndarray

In [18]:
print(data.shape)
print(data_target.shape)

(1000, 256, 256, 3)
(1000,)


In [29]:
data_target

array([         0,          0,          0,          0,          0,
                0,          1,          0,          1,          1,
                1,          1,          0,          1,          0,
                1,          1,          0,          0,          0,
                1,          0,          1,          1,          1,
                1,          0,          0,          1,          1,
                0,          1,          1,          0,          1,
                1,          1,          0,          1,          1,
                1,          0,          1,          1,          1,
                1,          1,          1,          0,          1,
                0,          0,          1,          0,          0,
                0,          0,          1,          1,          0,
                0,          1,          0,          0,          0,
                1,          1,          0,          0,          1,
                1,          0,          0,          0,        

In [19]:
#Split test and training data
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(data,data_target, test_size = 0.2)
x_train.shape, x_val.shape, y_train.shape, y_val.shape

((800, 256, 256, 3), (200, 256, 256, 3), (800,), (200,))

In [23]:
x_train

array([[[[ 11.,  47.,  63.],
         [  8.,  44.,  60.],
         [ 11.,  47.,  63.],
         ...,
         [  7.,  43.,  59.],
         [  6.,  42.,  58.],
         [  6.,  42.,  58.]],

        [[ 16.,  52.,  68.],
         [ 11.,  47.,  63.],
         [ 11.,  47.,  63.],
         ...,
         [  0.,  36.,  52.],
         [  7.,  43.,  59.],
         [  3.,  39.,  55.]],

        [[  8.,  44.,  60.],
         [  9.,  45.,  61.],
         [ 10.,  46.,  62.],
         ...,
         [  2.,  38.,  54.],
         [  3.,  39.,  55.],
         [  5.,  41.,  57.]],

        ...,

        [[  8.,  47.,  62.],
         [  6.,  45.,  60.],
         [  8.,  47.,  62.],
         ...,
         [  0.,  37.,  55.],
         [  3.,  40.,  58.],
         [  4.,  41.,  59.]],

        [[  8.,  47.,  62.],
         [  4.,  43.,  58.],
         [  7.,  46.,  61.],
         ...,
         [  5.,  42.,  60.],
         [  6.,  43.,  61.],
         [  4.,  41.,  59.]],

        [[  9.,  48.,  63.],
       

In [24]:
x_val

array([[[[ 24.,  66.,  65.],
         [ 30.,  72.,  71.],
         [ 26.,  68.,  67.],
         ...,
         [ 25.,  64.,  69.],
         [ 30.,  69.,  74.],
         [ 31.,  70.,  75.]],

        [[ 28.,  70.,  69.],
         [ 25.,  67.,  66.],
         [ 29.,  71.,  70.],
         ...,
         [ 34.,  73.,  78.],
         [ 28.,  67.,  72.],
         [ 30.,  69.,  74.]],

        [[ 32.,  74.,  73.],
         [ 24.,  66.,  65.],
         [ 35.,  77.,  76.],
         ...,
         [ 30.,  69.,  74.],
         [ 30.,  69.,  74.],
         [ 27.,  66.,  71.]],

        ...,

        [[ 32.,  76.,  77.],
         [ 30.,  74.,  75.],
         [ 26.,  70.,  71.],
         ...,
         [ 28.,  66.,  69.],
         [ 32.,  70.,  73.],
         [ 32.,  70.,  73.]],

        [[ 31.,  75.,  76.],
         [ 39.,  83.,  84.],
         [ 30.,  74.,  75.],
         ...,
         [ 37.,  75.,  78.],
         [ 28.,  66.,  69.],
         [ 32.,  70.,  73.]],

        [[ 31.,  75.,  76.],
       

In [25]:
y_train

array([0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0,

In [26]:
y_val

array([1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 0], dtype=int32)

In [21]:
#Set training and evaluation hyper paramenters
import tensorflow as tf
'''
x - according to the documentation I am passing a dictionary of my training features
y - passing the training labels
batch_size - how many images I want to pass to my NN in a single training step. 
             The gradients and the loss for the training step will be calculated only on these images. 
num_epochs - when I want to complete the execution of this method. With the current setting (1) 
             when the input function goes through all images once it will complete.
shuffle    - do I want to read the images in order or no. It is a better strategy to 
             shuffle within the training images during training.
'''
train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={'x':x_train},
      y=y_train,
      batch_size=20,
      num_epochs=2,
      shuffle=True)

'''
What the eval_input_fn will do with its current settings is to read the entire training dataset 
(in order) using batch_size of 128 (default setting). 

Keep in mind that it will also terminate when completes 1 epoch.
'''
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x':x_val},
    y=y_val,
    shuffle=False,
    num_epochs=2)



In [24]:
#Define CNN Model
def cnn_model_function(features, labels, mode):
    '''
    This is the function which describes the structure of the neural network
    '''
    
    # input layer
    # reshaping x to 4-D tensor: [batch_size, width, height, channels]
    # features['x'] - the dictionary we passed for x in the input functions
    layer_1 = tf.reshape(features['x'], [-1, 256, 256, 3])
    
    # convolution layer 1
    # computes 32 features using 10x10 filter with ReLU activation.
    # input tensor: [batch_size, 256, 256, 3]
    # output tensor: [batch_size, 256, 256, 32]
    layer_2 = tf.layers.conv2d(                          
                inputs=layer_1,
                filters=10,
                kernel_size=[10, 10],
                padding="same",
                activation=tf.nn.relu)
    
    # convolution layer 2
    # computes 32 features using 5x5 filter with ReLU activation.
    # input tensor: [batch_size, 256, 256, 32]
    # output tensor: [batch_size, 256, 256, 64]
    layer_3 = tf.layers.conv2d(
                inputs=layer_2,
                filters=20,
                kernel_size=[5, 5],
                padding="same",
                activation=tf.nn.relu)
    
    # Flatten tensor into a batch of vectors
    # Input Tensor Shape: [batch_size, 256, 256, 64]
    # Output Tensor Shape: [batch_size, 256 * 256 * 64]
    layer_4 = tf.reshape(layer_3, [-1, 256 * 256 * 20])
    
    # Dense Layer
    # Densely connected layer with 1024 neurons
    # Input Tensor Shape: [batch_size, 256 * 256 * 64]
    # Output Tensor Shape: [batch_size, 1024]
    layer_5 = tf.layers.dense(inputs=layer_4, units=20, activation=tf.nn.relu)
    
    # Dropout operation; 0.6 probability that element will be kept
    # notice that this layer will perform droupout only during training!
    layer_6 = tf.layers.dropout(inputs=layer_5, rate=0.2, training=(mode == tf.estimator.ModeKeys.TRAIN))
    
    # Logits layer
    # Input Tensor Shape: [batch_size, 1024]
    # Output Tensor Shape: [batch_size, 2]
    logits = tf.layers.dense(inputs=layer_6, units=2)
    
    print("Logits: %s"% (logits))
    
    # define the values which our neural network will output
    # classes - which number the NN 'thinks' is on the image
    # probabilities - how certain our NN is about its prediction
    predictions = {
                "classes": tf.argmax(input=logits, axis=1),
                "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    print("classes: %s - Predictions: %s"% (tf.argmax(input=logits, axis=1), tf.nn.softmax(logits, name="softmax_tensor")))
    
    # here we define what happens if we call the predict method of our estimator
    # with the current settings it will return the dictionary defined above
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    # here we define the loss for our training (the thing we minimize)
    # I do not need to perform one-hot-encoding to my training labels because the method
    # sparse_softmax_cross_entropy will do that for me and I don't need to think about that
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    # here we define how we calculate our accuracy
    # if you want to monitor your training accuracy you need these two lines
    accuracy = tf.metrics.accuracy(labels=labels, predictions=predictions['classes'], name='acc_op')
    tf.summary.scalar('accuracy', accuracy[1])
    
    # here we define what happens if we call the train method of our estimator
    # with its current settings it will adjust the weights and biases of our neurons
    # using the Adam Optimization Algorithm based on the loss function we defined earlier
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer()
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        tf.identity(accuracy[1], name='train_accuracy')
        tf.summary.scalar('train_accuracy', accuracy[1])
        eval_metric_ops = {"train_accuracy":accuracy}
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op, eval_metric_ops=eval_metric_ops)
    
    # what evaluation metric we want to show
    tf.identity(accuracy[1], name='val_accuracy')
    tf.summary.scalar('val_accuracy', accuracy[1])
    eval_metric_ops = {'val_accuracy':accuracy}
    # here we define what happens if we call the evaluate method of our estimator
    # with its current settings it will display the loss and the accuracy which we defined earlier
    return tf.estimator.EstimatorSpec(
                mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [25]:
tf.logging.set_verbosity(tf.logging.INFO)

In [26]:
# Define Tensorboard variables
# first we define a folder where tensorflow will keep its progress
# this includes periodical saves of our weight, biases, accuracy, loss and etc.
# so if we have more training images we can simply continue training on them
OUTDIR = './CNN_CLASSIFIER'

# we create an estimator object which:
# - is using the Neural Net structure from the cnn_model_function
# - reads/writes the files written in the directory which we defined earlier
cnn_classifier = tf.estimator.Estimator(model_fn=cnn_model_function, model_dir=OUTDIR)

# here we start the FileWriter method which will actually save the progress in the folder defined above
file_writer = tf.summary.FileWriter(OUTDIR)

# a small helper function which trains/evaluates our network for a given number of epochs
# remember that our input functions go through the datasets only once
def train_and_evaluate(estimator, epochs=2):
    for i in range(epochs):
        estimator.train(input_fn=train_input_fn)
        estimator.evaluate(input_fn=eval_input_fn)

# and here we finally start training/evaluating the NN for 30 epochs
train_and_evaluate(cnn_classifier)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './CNN_CLASSIFIER', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f63c2d8ac18>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, u

In [27]:
#Start importing test data
test_v2 = os.listdir('./test')
print(len(test_v2))
type(len(test_v2))

15607


int

In [28]:
test_v2

['f16445c27.jpg',
 '49f61e424.jpg',
 'f9eefd48e.jpg',
 'ab46ec437.jpg',
 '9f5d2d97b.jpg',
 'b051f2477.jpg',
 '84c0874df.jpg',
 '262b4a6ea.jpg',
 'c51f74f68.jpg',
 '54ea4c059.jpg',
 'f6b9cd464.jpg',
 'd7783c3f6.jpg',
 '3e3841345.jpg',
 '17e2e6a80.jpg',
 'aafcbfd36.jpg',
 'd27ca7ed7.jpg',
 '386629009.jpg',
 '56990455f.jpg',
 'f7dee303d.jpg',
 '179dcd096.jpg',
 '2ef0374eb.jpg',
 'c01269474.jpg',
 'f03ed22c2.jpg',
 '80594a517.jpg',
 '79c525dfb.jpg',
 '390ca1774.jpg',
 '1f0db10ad.jpg',
 '93093c09e.jpg',
 'dff867e20.jpg',
 '9a5f4656b.jpg',
 'a27f7c6e1.jpg',
 '249cb920f.jpg',
 'c0fde3337.jpg',
 '1a4d379c1.jpg',
 '75f592582.jpg',
 '2a66cbb50.jpg',
 '427f8e757.jpg',
 '582c4a048.jpg',
 '78a8782a6.jpg',
 'c466efb68.jpg',
 'e002ec2d5.jpg',
 '4fdeda409.jpg',
 '7f9dce9a6.jpg',
 '401ac932e.jpg',
 'b1ff2e24b.jpg',
 'a82bebb5d.jpg',
 'b43cfb162.jpg',
 'f42b7eaa7.jpg',
 'c848efa8d.jpg',
 '382876096.jpg',
 '9c7d86e96.jpg',
 '1ebfc0edf.jpg',
 '210284cb9.jpg',
 '7169836af.jpg',
 '6721d2c13.jpg',
 '843e06c1

In [29]:
# Import test images
from PIL import Image
test_data = np.empty((len(test_v2),256,256,3), dtype=np.float32)
test_data_target = np.empty((len(test_v2)), dtype=np.int32)
image_name_list = test_v2
index = 0
for image_name in image_name_list:
    if image_name in list(test_v2):
        imageA = Image.open('./test/' + image_name).resize((256,256)).convert('RGB')
        test_data[index]=imageA
        # prediction
        index+=1


OSError: cannot identify image file './test/events.out.tfevents.1542309718.instance-3'

In [ ]:
'''
What the predict_input_fn will do with its current settings is to read the entire inference dataset 
(in order) using batch_size of 128 (default setting). 

Keep in mind that it will also terminate when completes 1 epoch.
'''
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x':test_data},
    shuffle=False,
    num_epochs=2)

In [ ]:
test_data.shape

In [ ]:
len(test_data)

In [ ]:
generator = cnn_classifier.predict(input_fn=predict_input_fn)

In [ ]:
def unique(list1): 
  
    # intilize a null list 
    unique_list = [] 
      
    # traverse for all elements 
    for x in list1: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 
    # print list 
    for x in unique_list: 
        print (x, end=" ")

In [ ]:
#Start prediction
preds = []
b = []
for i in range(len(test_data)):
    a = next(generator)
    preds.append(a['classes'])
    b.append(i)

In [ ]:
unique(preds)

In [45]:
len(preds)

15606

In [46]:
#Create classifcation file (not submission)
sub = np.column_stack((test_v2, preds))
submission = pd.DataFrame(sub)
submission.columns = ['ImageId', 'exist_ship']
submission.to_csv("submissionv4_1.csv")

NameError: name 'preds' is not defined